In [71]:
!pip install "git+https://github.com/jemisjoky/TorchMPS.git"
!pip install "git+https://github.com/rballester/tntorch.git"
import tntorch as tn
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
!pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/jemisjoky/TorchMPS.git to c:\users\piche\appdata\local\temp\pip-req-build-hv9d1uxr
  Resolved https://github.com/jemisjoky/TorchMPS.git to commit 6c0bc1a8e2c15acba8570ca9ffe2b4a0c7135165
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/jemisjoky/TorchMPS.git 'C:\Users\piche\AppData\Local\Temp\pip-req-build-hv9d1uxr'


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/rballester/tntorch.git to c:\users\piche\appdata\local\temp\pip-req-build-jhq0iseq
  Resolved https://github.com/rballester/tntorch.git to commit 241bf7ad2b806f6677a5e23534247f35f3a70f10
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/rballester/tntorch.git 'C:\Users\piche\AppData\Local\Temp\pip-req-build-jhq0iseq'


Defaulting to user installation because normal site-packages is not writeable


We shall choose the Hyperparameters in our setup

In [72]:
# Hardware hyperparameters
chosen_device = torch.device('cuda' 
if torch.cuda.is_available() else 'cpu')


# Data hyperparameters
data_dim = [1,28,28] # size of the individual input images (channel size, dims)
nb_train = 4
nb_test = 4
chosen_bs = 2
nb_classes = 10

# Kernel parameters
kernel_size = 4*4
padding = 0
stride = 4

# Student training hyperparameters
# general settings
n_epochs_lmps = 18
mps_learn_rate = 0.0001
mps_reg = 0.0
student_loss = nn.CrossEntropyLoss().to(chosen_device)
# mps parameters
input_dim = kernel_size
output_dim = 1
feature_dim = 2
bond_dim = 20
adaptive_mode = False
periodic_bc = False
# gaussian noise parameters
gn_var = 0.3
gn_mean = 0


In [73]:
# Import the mnist dataset
train_set = torchvision.datasets.MNIST(root = './datasets', train = True,    transform = transforms.ToTensor(),  download = True )

train_subset = torch.utils.data.SubsetRandomSampler(range(nb_train))

train_iterator = torch.utils.data.DataLoader(dataset = train_set, sampler = train_subset, batch_size=chosen_bs)


test_set = torchvision.datasets.MNIST(root = './datasets', train = False, transform = transforms.ToTensor(),  download = True)

test_subset = torch.utils.data.SubsetRandomSampler(range(nb_test))

test_iterator = torch.utils.data.DataLoader(dataset = test_set, sampler = test_subset, batch_size = chosen_bs)

In [74]:

class LMPS_patcher(nn.Module):
    def __init__(self):
        super(LMPS_patcher, self).__init__()

        # Create the MPS instance for the student
        self.mps_student = MPS(
            input_dim = input_dim,
            feature_dim = feature_dim,
            output_dim = output_dim,
            bond_dim = bond_dim,
            adaptive_mode = adaptive_mode,
            periodic_bc = periodic_bc,
        )
        # we define our original Φ, which creates the multilinear feature space
        self.feature_map = lambda x : torch.tensor([1, x]).reshape(-1)
        # to make the mps use our feature map for the inputs
        self.mps_student.register_feature_map(self.feature_map)


        # Create the unfolder, which will create our patches
        # the output size is [batch_size, nb_patches, patch_size]
        self.folding_params = dict(kernel_size=kernel_size, 
            dilation=1, padding=padding, stride=stride)
        self.unfolder = torch.nn.Unfold(**self.folding_params)

        # Get the number of patches with a little trick
        fake_image = torch.zeros([1] + data_dim)
        unfold_output = self.unfolder(fake_image)
        nb_patches = unfold_output.size()[1]
        self.nb_patches = nb_patches

        # To store the outputs of the MPS over the patches
        self.mps_outputs = torch.zeros([chosen_bs, nb_patches])


        # Fully connected layer
        self.lin = nn.Linear(nb_patches, nb_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        patches = self.unfolder(x)
        mps_outputs = torch.zeros([chosen_bs, self.nb_patches])
        for i in range(chosen_bs):
            for j in range(self.nb_patches):
                # get patch tensor
                flattened_patch = patches[i,j,:].reshape((1, kernel_size))
                mps_outputs[i,j] = self.mps_student(flattened_patch)
        y = self.lin(mps_outputs)
        y = self.relu(y)
        return y


student = LMPS_patcher()

# Instantiate the optimizer and softmax
student_optimizer = torch.optim.Adam(student.parameters(), lr = mps_learn_rate,
                                  weight_decay = mps_reg)
softmax = nn.Softmax(dim=1)

# Training loop 
for epoch in range(n_epochs_lmps):
    for (x_mb, y_mb) in train_iterator:
        # Reshape the train_tuple and put on the chosen device
        x_mb = x_mb.to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Foward propagation
        y_hat_mb = student(x_mb)
        loss = student_loss(y_hat_mb, y_mb)
        # Backpropagation
        loss.backward(retain_graph=True)
        student_optimizer.step()
        student_optimizer.zero_grad()

    print(loss.item())


# get accuracy

for (x_mb, y_mb) in test_iterator:
    print(MulticlassAccuracy(num_classes=10)(student(x_mb), y_mb))


2.2074975967407227
2.4286603927612305
2.416226387023926
2.4094557762145996
2.4066452980041504
2.404113292694092
2.4018149375915527
2.399850606918335
2.3982415199279785
2.3964695930480957
2.0534210205078125
2.39890193939209
2.0431623458862305
2.4048984050750732
2.4079666137695312
2.026909828186035
2.413668155670166
2.016709089279175
testing student
tensor(0.1000)
testing student
tensor(0.)
